In [103]:
import yfinance as yf
import pandas as pd
import numpy as np
import vectorbt as vbt
import empyrical as ep
import warnings
from datetime import datetime, timedelta

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

In [104]:
TFEX_S50 = pd.read_excel("S50_TFEX.xlsx")

In [105]:
TFEX_S50

,Timestamp,S50H0^2,S50M0^2,S50U0^2,S50Z0^2,S50H1^2,S50M1^2,S50U1^2,S50Z1^2,S50H2^2,...,S50U2^2,S50Z2^2,S50H3^2,S50M3^2,S50U3^2,S50Z3^2,S50H4^2,S50M4^2,S50U4,S50Z4
0,NaT,Trade Close,Trade Close,Trade Close,Trade Close,Trade Close,Trade Close,Trade Close,Trade Close,Trade Close,...,Trade Close,Trade Close,Trade Close,Trade Close,Trade Close,Trade Close,Trade Close,Trade Close,Trade Close,Trade Close
1,2020-01-02,1076.6,1071.7,1065.6,1064.3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-03,1075.4,1070.1,1064.3,1063.2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-06,1059.2,1055,1048.9,1048.3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-07,1070.9,1066.2,1059.9,1059,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,2024-07-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,821,821.2
1100,2024-07-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,816.7,817
1101,2024-07-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,820.5,820.6
1102,2024-07-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,824.2,824


In [106]:
TFEX_S50['Timestamp'] = pd.to_datetime(TFEX_S50['Timestamp'])

TFEX_S50.set_index('Timestamp', inplace=True)

In [107]:
P24_H2 = TFEX_S50[['S50H0^2','S50M0^2']]
P24_H2 = P24_H2.dropna()
P24_H2

,S50H0^2,S50M0^2
Timestamp,,
NaT,Trade Close,Trade Close
2020-01-02,1076.6,1071.7
2020-01-03,1075.4,1070.1
2020-01-06,1059.2,1055
2020-01-07,1070.9,1066.2
...,...,...
2020-03-24,696.1,688.1
2020-03-25,723.9,717.5
2020-03-26,737.7,729.2


In [108]:
cols = P24_H2.columns

first_col = cols[0]
second_col = cols[1]

In [109]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2[first_col], mode='lines', name=first_col, line=dict(color='blue')))
fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2[second_col], mode='lines', name=second_col, line=dict(color='red')))

fig.update_layout(
    title=f'{first_col} and {second_col} Over Time',
    xaxis_title='Date',
    yaxis_title='Value',
    template='plotly_dark'
)
fig.show()

In [110]:
P24_H2_CORR = P24_H2.iloc[0:51]
P24_H2_TEST = P24_H2.iloc[1:]

In [111]:
P24_H2_CORR

,S50H0^2,S50M0^2
Timestamp,,
NaT,Trade Close,Trade Close
2020-01-02,1076.6,1071.7
2020-01-03,1075.4,1070.1
2020-01-06,1059.2,1055
2020-01-07,1070.9,1066.2
2020-01-08,1054.2,1050.1
2020-01-09,1068,1063.3
2020-01-10,1068.7,1063.8
2020-01-13,1071.8,


In [112]:
P24_H2_CORR = P24_H2.iloc[0:51]
P24_H2_TEST = P24_H2.iloc[1:]

In [113]:
P24_H2[first_col] = pd.to_numeric(P24_H2[first_col], errors='coerce')
P24_H2[second_col] = pd.to_numeric(P24_H2[second_col], errors='coerce')

In [114]:
P24_H2['Gap'] = P24_H2[first_col] - P24_H2[second_col]
window = 20  # Adjust the window size
P24_H2['Rolling Mean'] = P24_H2['Gap'].rolling(window=window).mean()
P24_H2['Rolling Std'] = P24_H2['Gap'].rolling(window=window).std()
P24_H2['Z-score'] = (P24_H2['Gap'] - P24_H2['Rolling Mean']) / P24_H2['Rolling Std']
P24_H2['EMA'] = P24_H2['Gap'].ewm(span=window, adjust=False).mean()

In [115]:
fig = make_subplots(
    rows=6, cols=1,
    shared_xaxes=True,
    subplot_titles=(
        f'Price of {first_col}', 
        f'Price of {second_col}', 
        'Gap', 
        'Rolling Mean', 
        'EMA', 
        'Rolling Std and Z-Score'
    ),
    vertical_spacing=0.1
)

fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2[first_col], mode='lines', name=first_col, line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2[second_col], mode='lines', name=second_col, line=dict(color='orange')), row=2, col=1)
fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2['Gap'], mode='lines', name='Gap', line=dict(color='purple')), row=3, col=1)
fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2['Rolling Mean'], mode='lines', name='Rolling Mean', line=dict(color='green')), row=4, col=1)
fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2['EMA'], mode='lines', name='EMA', line=dict(color='orange', dash='dot')), row=5, col=1)
fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2['Rolling Std'], mode='lines', name='Rolling Std', line=dict(color='red')), row=6, col=1)
fig.add_trace(go.Scatter(x=P24_H2.index, y=P24_H2['Z-score'], mode='lines', name='Z-Score', line=dict(color='cyan')), row=6, col=1)

fig.update_layout(
    title=f'Prices, Gap, Rolling Statistics, EMA, and Z-Score between {first_col} and {second_col}',
    xaxis_title='Date',
    yaxis_title='Value',
    height=1800,
    margin=dict(t=100, b=100, l=80, r=80),
    template='plotly_dark'
)
fig.show()

In [116]:
entry_long = P24_H2['Z-score'] > 0.5
entry_short = P24_H2['Z-score'] < -0.5
exit_long_short = abs(P24_H2['Z-score']) < 0.0125

# Backtest with vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=P24_H2[first_col],
    entries=entry_long,
    short_entries=entry_short,
    exits=exit_long_short,
    short_exits=exit_long_short,
    direction='both'
)

# Output the performance
print(portfolio.stats())

Start                                         NaT
End                           2020-03-30 00:00:00
Period                                         63
Start Value                                 100.0
End Value                               68.467961
Total Return [%]                       -31.532039
Benchmark Return [%]                   -32.068549
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                        34.643066
Max Drawdown Duration                        33.0
Total Trades                                    3
Total Closed Trades                             2
Total Open Trades                               1
Open Trade PnL                          -0.594478
Win Rate [%]                                  0.0
Best Trade [%]                          -1.906341
Worst Trade [%]                        -29.595409
Avg Winning Trade [%]                         NaN
Avg Losing Trade [%]                   -15.750875


C:\Users\HP Victus 16\AppData\Local\Temp\ipykernel_2288\3246804033.py:6: UserWarning:

direction has no effect if short_entries and short_exits are set

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

c:\Users\HP Victus 16\AppData\Local\Programs\Python\Python311\Lib\site-packages\vectorbt\generic\stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



In [117]:
fig = portfolio.plot()
fig.update_layout(title='Trading Strategy Performance', xaxis_title='Date', yaxis_title='Portfolio Value')
fig.show()